In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/creditcardfraud/creditcard.csv


In [2]:
credit_data= pd.read_csv('/kaggle/input/creditcardfraud/creditcard.csv')

In [3]:
credit_data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
credit_data['Class'].value_counts()
#we can see here this data set is highly unbalanced and  our model will give wrong output so we have to preprocess

Class
0    284315
1       492
Name: count, dtype: int64

In [5]:
#seperating the data for analysis into normal and fraudful transaction
legit=credit_data[credit_data.Class==0]
fraud=credit_data[credit_data.Class==1]

In [6]:
#mean of fraud transaction is v high than the normal transaction
legit.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [7]:
fraud.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [8]:
#compare the mean value for both the transaction
credit_data.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [9]:
#dealing with unbalanced data and using the concept of undersampling
#build sample data set from original data set containing similar distribution of normal/legit transaction and fraudulent transaction
#no of fraud transaction =492

legit_sample=legit.sample(n=492)



In [10]:
#concatenate two data frame 
new_dataset=pd.concat([legit_sample,fraud],axis=0)

In [11]:
new_dataset.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
223464,143395.0,2.024433,0.281553,-1.673993,1.180634,0.756858,-0.611676,0.604127,-0.316391,-0.212609,...,0.088968,0.526617,-0.062302,-0.358147,0.514388,-0.489694,-0.009779,-0.074327,1.00,0
252472,155821.0,2.015437,-0.078327,-1.277559,0.207017,0.123341,-0.810154,0.172205,-0.186899,0.423248,...,-0.210148,-0.558933,0.249726,-0.381802,-0.144358,-0.604379,-0.022057,-0.061020,13.00,0
234750,148093.0,1.951753,-0.406073,-1.541865,0.938644,-0.326524,-0.537708,-0.562685,-0.010978,-0.174408,...,-0.421606,-0.795625,0.218032,0.441092,-0.207901,-0.706410,0.068936,0.032941,72.00,0
22739,32408.0,-0.209805,1.061183,1.126815,-0.168620,0.708735,-0.479472,0.922509,-0.287501,-0.327616,...,-0.358238,-0.743171,-0.167716,-0.480901,-0.042931,0.092453,0.086636,-0.130672,0.89,0
225119,144114.0,2.015838,-0.253886,-1.092709,-0.104475,0.111142,-0.161767,-0.237091,0.068763,0.409767,...,-0.436825,-1.399534,0.533022,0.192636,-0.723433,-0.000899,-0.080769,-0.055783,14.31,0


In [12]:
new_dataset.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,93158.587398,0.078449,-0.007897,0.046269,-0.053392,-0.021691,-0.035600,-0.075718,0.010781,0.044329,...,-0.019078,-0.043575,-0.034611,-0.033064,0.027753,-0.008947,-0.053995,0.019533,0.006353,76.758394
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [13]:
X=new_dataset.drop(columns='Class',axis=1)
Y=new_dataset['Class']

In [14]:
print(X)

            Time        V1        V2        V3        V4        V5        V6  \
223464  143395.0  2.024433  0.281553 -1.673993  1.180634  0.756858 -0.611676   
252472  155821.0  2.015437 -0.078327 -1.277559  0.207017  0.123341 -0.810154   
234750  148093.0  1.951753 -0.406073 -1.541865  0.938644 -0.326524 -0.537708   
22739    32408.0 -0.209805  1.061183  1.126815 -0.168620  0.708735 -0.479472   
225119  144114.0  2.015838 -0.253886 -1.092709 -0.104475  0.111142 -0.161767   
...          ...       ...       ...       ...       ...       ...       ...   
279863  169142.0 -1.927883  1.125653 -4.518331  1.749293 -1.566487 -2.010494   
280143  169347.0  1.378559  1.289381 -5.004247  1.411850  0.442581 -1.326536   
280149  169351.0 -0.676143  1.126366 -2.213700  0.468308 -1.120541 -0.003346   
281144  169966.0 -3.113832  0.585864 -5.399730  1.817092 -0.840618 -2.943548   
281674  170348.0  1.991976  0.158476 -2.583441  0.408670  1.151147 -0.096695   

              V7        V8        V9  .

In [15]:
print(Y)

223464    0
252472    0
234750    0
22739     0
225119    0
         ..
279863    1
280143    1
280149    1
281144    1
281674    1
Name: Class, Length: 984, dtype: int64


In [16]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2, stratify=Y, random_state=2)

In [17]:
print(X.shape,X_train.shape,X_test.shape)

(984, 30) (787, 30) (197, 30)


In [18]:
model = LogisticRegression(max_iter=2200) 

In [19]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=2200)

In [20]:
X_train_prediction= model.predict(X_train)
training_data_accuracy=accuracy_score(X_train_prediction,Y_train)

In [21]:
print('accuracy of our test data',training_data_accuracy)

accuracy of our test data 0.9275730622617535


In [22]:
X_test_prediction= model.predict(X_test)
test_data_accuracy=accuracy_score(X_test_prediction,Y_test)

In [23]:
print('accuracy of our test data',test_data_accuracy)

accuracy of our test data 0.9086294416243654


In [24]:
res=X_test.copy()
res['actual']=Y_test
res['prediction']=X_test_prediction

res.to_csv('submission.csv',index=False)